###Feature Engineering for Natural Language Processing; Classification and Clustering

####Sentiment Analysis with Amazon Review Dataset

In [3]:
display(spark.read.parquet("/databricks-datasets/amazon/data20K"))

rating,review
4.0,Worked as expected.I'm not sure what else you expect me to say. I expected no less.Dunno what else to say.
5.0,"This mouse is amazing, I had owned a Razer Naga, and a R.A.T. 7 mouse. And I am not afraid to say that this mouse takes the cake, for $20 it looks like its worth $80. The only issue i had was that the seller sent me a wireless version, but I don't mind because it works just as well. Also where you see silver on the mouse in reality it is glossy black, and the mouse looks better IRL then in the pictures. I will add on to this review in a month or so to check in."
4.0,we recently had a baby boy so now the use of my home theater system is on the shelf for awhile. My wife also is not into big sound so decided to try out these wireless headphones. at first i almost sent them back because i took the easy route and hooked them into my receivers headphone jack. The sound was terrible and there was the dreaded hissing noise you get with some wireless electronics. before packing them up i decided to hook them up into the jacks on the TV. Difference was like night and day. the sound quality is great and really brings out some of the background sounds and music that you typically don't notice unless you are in a movie theater. They aren't super comfortable for long durations but that can be expected with most headphones. these come fully recommended.
3.0,"Works good for a boy of 6 and his parents.This game is not really requiring any skills (be it game skills, memory or else). It is still fun but gets repetitive fast. Said that, my son still enjoyed it enough to want a proper ""adult"" Catan."
2.0,"Fabric is nice and soft but zipper broke the first time we used it. Very disappointing. The fit was fine, so we still use it to get our monies worth"
4.0,"They are screws with square heads, what else do you need to know. They work, they hold two pieces of wood together."
5.0,I love this prolduct and the other items that go with it. Bath and Body dropped this scent so I as thrilled find that I could still purchase it.
3.0,"This is an ok pump. It has the nice bracket that can mount to your bike and you have it for the road.And although the image here doesn't show it, it does have the gauge, though it isn't the most accurate gauge (not really a problem when the tire is rated for 55 to 100 PSI).Much like one other reviewer noted, the instructions on how to switch from Schrader to Presta are not very clear and should be researched before fumbling with this blasted thing for hours trying to make it work.And like the title mentions, this isn't as tiny as the MINI name implies. When looking for a place to mount it on my bike, I am really running short on ideas. The obvious places would interfere with pedaling.All that said, it puts air in the tire. Slowly. It is double action, but the throw isn't very much. And the higher the pressure, the more difficult it is to pump."
5.0,"Perfect for the gym or under a few layers for the chilly morning runs. I own quite a few, and just can't get enough!"
4.0,"I'd hoped my 2 year old kitty would have been more enthusiastic about these little Chew Mice, but perhaps she has too many other toys. All-in-all, they're a good addition to her toy collection."


Distribution of Ratings vs. Distribution of Ratings Containing Certain Words...

In [5]:
raw = spark.read.parquet("/databricks-datasets/amazon/data20K").withColumnRenamed("rating", "label")

In [6]:
display(raw.groupBy("label").count().orderBy("label"))

label,count
1.0,1881
2.0,1138
3.0,1696
4.0,3460
5.0,11784


The negative reviews are substantially underrepresented here. If we're not careful, we'll train a model that thinks "Everything is Awesome!" and is usually right for this dataset. Let's oversample the negative reviews and see how that looks:

In [8]:
more_balanced = raw.filter("label < 4").sample(True, 3.0).union(raw)

In [9]:
display(more_balanced.groupBy("label").count().orderBy("label"))

label,count
1.0,7538
2.0,4594
3.0,6838
4.0,3460
5.0,11784


In [10]:
display(more_balanced.filter('review LIKE "%awesome%"').groupBy("label").count().orderBy("label"))

label,count
1.0,33
2.0,31
3.0,89
4.0,49
5.0,266


In [11]:
display(more_balanced.filter('review LIKE "%poor%"').groupBy("label").count().orderBy("label"))

label,count
1.0,315
2.0,204
3.0,109
4.0,23
5.0,45


#####Not too surprising, eh? 

So the idea here is to try and build a model that can decide the positive or negative "sentiment" of a review (or text in general) based on the presence and frequency of certain words.

__In this example we're going to use "bag of words" model and count them in a way that emphasizes more common words. The hypothesis there is that rare words are more likely about the product under review, whereas day-to-day sentiment-signalling words should occur in a large subset of the documents.__

To prepare the features, we will split the text into a series of tokens...

... remove some noise in the form of "stop words"

... and create a "count vector" with counts of common words for each document.

In [13]:
from pyspark.ml.feature import *

tokenizer = RegexTokenizer(inputCol="review", outputCol="tokens", pattern="\\W+")

remover = StopWordsRemover(inputCol="tokens", outputCol="stopWordFree")

counts = CountVectorizer(inputCol="stopWordFree", outputCol="features",
                          vocabSize=1000, minDF=200)
# most frequent words subject to min doc occurrences ... (alt to Hashing TF / IDF)

#####Sidebar: Other NLP Featurization Approaches
* Instead of bag-of-words, you may want to extract named entities, part-of-speech, lemma, grammatical dependencies, etc. and use those as features
  * In the JVM world we have (examples)
    * Stanford NLP https://nlp.stanford.edu/software/
    * Spark NLP / John Snow Labs https://nlp.johnsnowlabs.com/
  * In the Python world
    * spaCy https://spacy.io/
    * NLTK https://www.nltk.org/
    * In Part 2 of this session, we'll look at how you can performantly integrate Spark with your favorite Python tools!
* Word Embeddings
  * Spark can also train its own embeddings https://spark.apache.org/docs/latest/ml-features.html#word2vec
  * Or you can use NLP libs like spaCy
  * Or deep learning libs like TensorFlow/PyTorch

__Let's build and fit a pipeline__

*Note: we aren't trying to build a model here (or make predictions). This is just a featurizing pipeline.*

Since Pipeline is an Estimator, it could be put inside another Pipeline ... so one pattern might be to use a Pipeline just for some feature engineering tasks.

In [16]:
from pyspark.ml import Pipeline

p1 = Pipeline(stages=[tokenizer, remover, counts])

In [17]:
data = p1.fit(raw).transform(raw)
display(data) # note the sparse vectors

label,review,tokens,stopWordFree,features
4.0,Worked as expected.I'm not sure what else you expect me to say. I expected no less.Dunno what else to say.,"List(worked, as, expected, i, m, not, sure, what, else, you, expect, me, to, say, i, expected, no, less, dunno, what, else, to, say)","List(worked, expected, m, sure, else, expect, say, expected, less, dunno, else, say)","List(0, 544, List(22, 79, 85, 113, 135, 213, 334, 424), List(1.0, 1.0, 2.0, 1.0, 1.0, 2.0, 2.0, 1.0))"
5.0,"This mouse is amazing, I had owned a Razer Naga, and a R.A.T. 7 mouse. And I am not afraid to say that this mouse takes the cake, for $20 it looks like its worth $80. The only issue i had was that the seller sent me a wireless version, but I don't mind because it works just as well. Also where you see silver on the mouse in reality it is glossy black, and the mouse looks better IRL then in the pictures. I will add on to this review in a month or so to check in.","List(this, mouse, is, amazing, i, had, owned, a, razer, naga, and, a, r, a, t, 7, mouse, and, i, am, not, afraid, to, say, that, this, mouse, takes, the, cake, for, 20, it, looks, like, its, worth, 80, the, only, issue, i, had, was, that, the, seller, sent, me, a, wireless, version, but, i, don, t, mind, because, it, works, just, as, well, also, where, you, see, silver, on, the, mouse, in, reality, it, is, glossy, black, and, the, mouse, looks, better, irl, then, in, the, pictures, i, will, add, on, to, this, review, in, a, month, or, so, to, check, in)","List(mouse, amazing, owned, razer, naga, r, 7, mouse, afraid, say, mouse, takes, cake, 20, looks, like, worth, 80, issue, seller, sent, wireless, version, mind, works, well, also, see, silver, mouse, reality, glossy, black, mouse, looks, better, irl, pictures, add, review, month, check)","List(0, 544, List(2, 6, 14, 17, 27, 60, 85, 102, 116, 223, 243, 268, 289, 294, 303, 313, 342, 345, 423, 470, 481), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))"
4.0,we recently had a baby boy so now the use of my home theater system is on the shelf for awhile. My wife also is not into big sound so decided to try out these wireless headphones. at first i almost sent them back because i took the easy route and hooked them into my receivers headphone jack. The sound was terrible and there was the dreaded hissing noise you get with some wireless electronics. before packing them up i decided to hook them up into the jacks on the TV. Difference was like night and day. the sound quality is great and really brings out some of the background sounds and music that you typically don't notice unless you are in a movie theater. They aren't super comfortable for long durations but that can be expected with most headphones. these come fully recommended.,"List(we, recently, had, a, baby, boy, so, now, the, use, of, my, home, theater, system, is, on, the, shelf, for, awhile, my, wife, also, is, not, into, big, sound, so, decided, to, try, out, these, wireless, headphones, at, first, i, almost, sent, them, back, because, i, took, the, easy, route, and, hooked, them, into, my, receivers, headphone, jack, the, sound, was, terrible, and, there, was, the, dreaded, hissing, noise, you, get, with, some, wireless, electronics, before, packing, them, up, i, decided, to, hook, them, up, into, the, jacks, on, the, tv, difference, was, like, night, and, day, the, sound, quality, is, great, and, really, brings, out, some, of, the, background, sounds, and, music, that, you, typically, don, t, notice, unless, you, are, in, a, movie, theater, they, aren, t, super, comfortable, for, long, durations, but, that, can, be, expected, with, most, headphones, these, come, fully, recommended)","List(recently, baby, boy, use, home, theater, system, shelf, awhile, wife, also, big, sound, decided, try, wireless, headphones, first, almost, sent, back, took, easy, route, hooked, receivers, headphone, jack, sound, terrible, dreaded, hissing

Split for testing

In [19]:
train, test = data.randomSplit([0.8, 0.2])

train = train.union(train.filter("label < 4").sample(True, 3.0))

Now we'll fit the model. While it's training, take a look at the Storage tab in the Web UI

In [21]:
from pyspark.ml.classification import *
from pyspark.ml.evaluation import *

mlor = LogisticRegression()
lrModel = mlor.fit(train)

Before we look at our test set performance, let's see some of the stats we can get from the training summary:

In [23]:
# Print the coefficients and intercept for multinomial logistic regression
print("Coefficients: \n" + str(lrModel.coefficientMatrix))
print("Intercept: " + str(lrModel.interceptVector))

trainingSummary = lrModel.summary

# Obtain the objective per iteration
objectiveHistory = trainingSummary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)

# for multiclass, we can inspect metrics on a per-label basis
print("False positive rate by label:")
for i, rate in enumerate(trainingSummary.falsePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

print("True positive rate by label:")
for i, rate in enumerate(trainingSummary.truePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

print("Precision by label:")
for i, prec in enumerate(trainingSummary.precisionByLabel):
    print("label %d: %s" % (i, prec))

print("Recall by label:")
for i, rec in enumerate(trainingSummary.recallByLabel):
    print("label %d: %s" % (i, rec))

print("F-measure by label:")
for i, f in enumerate(trainingSummary.fMeasureByLabel()):
    print("label %d: %s" % (i, f))

accuracy = trainingSummary.accuracy
falsePositiveRate = trainingSummary.weightedFalsePositiveRate
truePositiveRate = trainingSummary.weightedTruePositiveRate
fMeasure = trainingSummary.weightedFMeasure()
precision = trainingSummary.weightedPrecision
recall = trainingSummary.weightedRecall
print("Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
      % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))

Coefficients: 
DenseMatrix([[-0.00526035, -0.00670925, -0.00673562, ..., -0.00817839,
 -0.00977735, -0.00908199],
 [ 0.05518626, -0.85599651, -0.09720865, ..., 0.4738807 ,
 -1.050604 , -0.32419438],
 [ 0.02712073, -0.37530515, -0.02620647, ..., 0.23795711,
 0.00546587, -0.42547346],
 [-0.13427566, -0.06905434, 0.04282236, ..., -0.10514406,
 -0.05443584, 0.3771555 ],
 [-0.02318555, 0.54008659, 0.14586332, ..., -0.45824609,
 0.48862859, 0.20789626],
 [ 0.08041458, 0.76697867, -0.05853494, ..., -0.14026928,
 0.62072272, 0.17369808]])
Intercept: [-7.1112049095,1.81304467167,1.13824655207,1.53637835714,0.705424666679,1.91811066194]
objectiveHistory:
1.5237624978062918
1.300269734582019
1.1700600795514429
1.133568086284647
1.1183505139872278
1.0976440972989476
1.064436601474518
1.0507183648428404
1.0430641322260856
1.0354833673575092
1.0314299936524152
1.0223745304597287
1.0203012722216762
1.0182997366360995
1.0165593339245673
1.0152367496023722
1.0144360188109394
1.0138865122145806
1.013338062810184
1.0128728826547733
1.012637149734839
1.0123569770867837
1.0122459538771933
1.0120219494992821
1.011961661670251
1.0118637582315428
1.011758104361569
1.0117027142383852
1.0116574320716722
1.011619567082473
1.0115512442270702
1.0115202613455423
1.011491819479807
1.011476262967191
1.0114642512519985
1.01145400108808
1.011437250332591
1.0114219610756063
1.011409243180849
1.0114026622988994
1.0113943457822088
1.0113877564485747
1.0113777867355587
1.0113689263015377
1.0113669803797647
1.0113565782651115
1.011354538636649
1.0113503452887083
1.0113488773967274
1.0113466137653981
1.011343936193819
1.0113425531238296
1.0113413648503962
1.011340088064422
1.0113391933742022
1.0113380702740904
1.0113368769975248
1.011336159205548
1.0113353426395728
1.0113348092038785
1.011334116613056
1.011333819502652
1.0113335421679512
1.01133300647655
1.0113328938455408
1.0113324658057075
1.0113322803690787
1.011332064267371
1.0113318992253721
1.0113316401669572
1.0113314540365606
1.0113313104527568
1.0113310354834963
1.0113309270417326
1.0113307793036999
1.0113306739314285
1.0113305305459654
1.0113304941260122
1.0113303686143353
1.0113303163085428
1.0113302631105352
1.0113301916726454
1.0113301514461919
1.0113300697663503
1.011329998435961
1.0113299420391484
1.0113298916802766
1.0113298347306672
1.0113297881745253
1.0113297252751434
1.011329660513832
1.0113296148020652
False positive rate by label:
label 0: 0.11805980692253355
label 1: 0.04671280276816609
label 2: 0.11410589683951293
label 3: 0.014098854977384785
label 4: 0.22659604126010593
True positive rate by label:
label 0: 0.73454664699131
label 1: 0.3602860286028603
label 2: 0.5470686147586462
label 3: 0.13175796634443251
label 4: 0.8183849345675072
Precision by label:
label 0: 0.6411907828824961
label 1: 0.5419942076954902
label 2: 0.5417582417582417
label 3: 0.5154061624649859
label 4: 0.6543041850969717
Recall by label:
label 0: 0.73454664699131
label 1: 0.3602860286028603
label 2: 0.5470686147586462
label 3: 0.13175796634443251
label 4: 0.8183849345675072
F-measure by label:
label 0: 0.6847012073972184
label 1: 0.43284321823888977
label 2: 0.544400478512929
label 3: 0.20986598232107215
label 4: 0.7272039706925076
Accuracy: 0.6149118314187458
FPR: 0.1344852341864492
TPR: 0.6149118314187458
F-measure: 0.5895415892995274
Precision: 0.5999829803416662
Recall: 0.6149118314187458

And now a quick check on the test-set performance:

In [25]:
result = lrModel.transform(test)
predictionAndLabels = result.select("prediction", "label")

In [26]:
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Accuracy: %f" % evaluator.evaluate(predictionAndLabels))

Accuracy: 0.575567

That's significant but maybe we can do better if we simplify the problem a little ... 

Above, we're asking the network to discern 1-star vs. 2-star reviews, and 4-vs-5-star ones without all that much data.

What if we just try to split a "positive" vs. a "negative" sentiment class?

In [28]:
binarizer = Binarizer(inputCol="label", outputCol="binarized_label", threshold=3.5)

In [29]:
training_with_binarizer = binarizer.transform(train)
test_with_binarizer = binarizer.transform(test)

In [30]:
lr = LogisticRegression(labelCol="binarized_label")
model = lr.fit(training_with_binarizer)

result = model.transform(test_with_binarizer)

In [31]:
display(result)

label,review,tokens,stopWordFree,features,binarized_label,rawPrediction,probability,prediction
1.0,"50 cent is the most garbage rapper ever period, he has no poetic jesture, no lyrical content. after destroying ja rules career he comes out wif songs like lil bit and candy shop is real sad. i dont noe why 50 cent even went at nas. hes not even on his, level nas will kill him just like he killed jayz. if you really listen to rap and ur not no fake p*ssy u would not buy this garabge album. not one song in this album is classic it has a couple of catchy songs. 2pac is the best rapper to pick up the mic, a legend in his own rhymes. buy his album and dont even consider this trash album trust me.","List(50, cent, is, the, most, garbage, rapper, ever, period, he, has, no, poetic, jesture, no, lyrical, content, after, destroying, ja, rules, career, he, comes, out, wif, songs, like, lil, bit, and, candy, shop, is, real, sad, i, dont, noe, why, 50, cent, even, went, at, nas, hes, not, even, on, his, level, nas, will, kill, him, just, like, he, killed, jayz, if, you, really, listen, to, rap, and, ur, not, no, fake, p, ssy, u, would, not, buy, this, garabge, album, not, one, song, in, this, album, is, classic, it, has, a, couple, of, catchy, songs, 2pac, is, the, best, rapper, to, pick, up, the, mic, a, legend, in, his, own, rhymes, buy, his, album, and, dont, even, consider, this, trash, album, trust, me)","List(50, cent, garbage, rapper, ever, period, poetic, jesture, lyrical, content, destroying, ja, rules, career, comes, wif, songs, like, lil, bit, candy, shop, real, sad, dont, noe, 50, cent, even, went, nas, hes, even, level, nas, kill, like, killed, jayz, really, listen, rap, ur, fake, p, ssy, u, buy, garabge, album, one, song, album, classic, couple, catchy, songs, 2pac, best, rapper, pick, mic, legend, rhymes, buy, album, dont, even, consider, trash, album, trust)","List(0, 544, List(0, 2, 9, 20, 25, 45, 75, 136, 162, 201, 208, 298, 518), List(1.0, 2.0, 1.0, 3.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0))",0.0,"List(1, 2, List(), List(0.5925827940994242, -0.5925827940994242))","List(1, 2, List(), List(0.6439575389106098, 0.3560424610893902))",0.0
1.0,"After following the ""easy"" installation steps, I could not, for the life of me, get my computer to connect to the router despite clicking ""Connect"" to the SSID hundreds of times on my computer. I have tried with two other computers and my iPhone but to no avail. Probably I was one of the unlucky ones that ended up with a defected refurbished product but hey, if it's not even working I'm not generous enough to give it a second try.","List(after, following, the, 34, easy, 34, installation, steps, i, could, not, for, the, life, of, me, get, my, computer, to, connect, to, the, router, despite, clicking, 34, connect, 34, to, the, ssid, hundreds, of, times, on, my, computer, i, have, tried, with, two, other, computers, and, my, iphone, but, to, no, avail, probably, i, was, one, of, the, unlucky, ones, that, ended, up, with, a, defected, refurbished, product, but, hey, if, it, s, not, even, working, i, m, not, generous, enough, to, give, it, a, second, try)","List(following, 34, easy, 34, installation, steps, life, get, computer, connect, router, despite, clicking, 34, connect, 34, ssid, hundreds, times, computer, tried, two, computers, iphone, avail, probably, one, unlucky, ones, ended, defected, refurbished, product, hey, even, working, m, generous, enough, give, second, try)","List(0, 544, List(0, 5, 7, 15, 20, 22, 33, 36, 61, 104, 125, 130, 133, 176, 193, 220, 237, 265, 279, 397), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 4.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0))",0.0,"List(1, 2, List(), List(1.48457880930655, -1.48457880930655))","List(1, 2, List(), List(0.8152631868652388, 0.18473681313476112))",0.0
1.0,Battery was useless of about 2 months of use. Will longer hold a charge at all. Would not recommend buying.,"List(battery, was, useless, of, about, 2, month

In [32]:
evaluator = MulticlassClassificationEvaluator(metricName="accuracy", labelCol="binarized_label")
print("Accuracy: %f" % evaluator.evaluate(result))

Accuracy: 0.751637

Not terrible. Let's introduce a BinaryClassificationEvaluator, which can calculate things like AUC

In [34]:
evaluator = BinaryClassificationEvaluator(labelCol="binarized_label")
print("AUC: %f" % evaluator.evaluate(result))

AUC: 0.835013

Ok... Let's look at accuracy and an ROC curve

In [36]:
display(model, test_with_binarizer, "ROC")

False Positive Rate,True Positive Rate,Threshold
0.0,0.0,0.9981207742401468
0.0,0.0125,0.9981207742401468
0.0,0.025,0.9945677247425737
0.0,0.0375,0.9936579487578436
0.0,0.05,0.9907917279098176
0.0,0.0625,0.9900043792504191
0.0,0.075,0.9877624578761379
0.0,0.0875,0.9872181261058103
0.0,0.1,0.9857378349799154
0.0,0.1125,0.9848298939087085


Let's try and tune with cross-validation. 

We'd like to wrap the Cross Validator around the entire pipeline, so we can have Spark perform grid search on our featurizers (like CountVectorizer) as well as the logistic regression itself. 

So we'll set this up as an end-to-end pipeline:

In [38]:
from pyspark.ml.tuning import *

raw = spark.read.parquet("/databricks-datasets/amazon/data20K").withColumnRenamed("rating", "label")

train, test = raw.randomSplit([0.8, 0.2])
train = train.union(train.filter("label < 4").sample(True, 3.0)).cache()

paramGrid = ParamGridBuilder() \
            .addGrid(counts.vocabSize, [300, 600]) \
            .addGrid(lr.regParam, [0.01, 0.05]) \
            .build()

cvpipeline = Pipeline(stages=[tokenizer, remover, counts, binarizer, lr])

cv = CrossValidator(estimator=cvpipeline, evaluator=evaluator, 
                        estimatorParamMaps=paramGrid, numFolds=3)

cvModel = cv.fit(train)

How did the different param sets do?

In [40]:
cvModel.avgMetrics

Out[ 24 ]: 
[0.8433811019806017,
 0.8738975718527251,
 0.8436331447969926,
 0.8747221469545872]

When using CrossValidator, `bestModel` is trained by CrossValidator over the whole training set

In [42]:
cvModel.bestModel

Out[ 25 ]: PipelineModel_309d8808b3d4

####Clustering 
__An Experiment: Do the reviews (or their feature vectors) cluster by sentiment?__

In [44]:
from pyspark.ml.clustering import *

kmeans = KMeans(k=10)
model = kmeans.fit(data)

In [45]:
WSSSE = model.computeCost(data)
print("Within Set Sum of Squared Errors = %f" % WSSSE)

Within Set Sum of Squared Errors = 478621.957164

In [46]:
model.clusterCenters()[0]

Out[ 28 ]: 
array([ 0.10872894, 0. , 0. , 0.19190276, 0.15811639,
 0.17161179, 0.15055513, 0.11102603, 0.10394334, 0.08346095,
 0.12050153, 0.08192956, 0.08441807, 0.09609495, 0.06163859,
 0.09676493, 0.09054364, 0.08250383, 0.08709801, 0.08221669,
 0.06431853, 0.05733155, 0.05465161, 0.05589587, 0.07810107,
 0.07714395, 0.049866 , 0.06355283, 0.0601072 , 0.05178025,
 0.06183002, 0.0632657 , 0.07551685, 0.00488132, 0.04804747,
 0.06766845, 0.04536753, 0.04680322, 0.03407351, 0.04029479,
 0.04345329, 0.049866 , 0.05158882, 0.05254594, 0.04335758,
 0.05388591, 0.04335758, 0.04785605, 0.03627489, 0.03436064,
 0.05790582, 0.03570061, 0.04546325, 0.03924196, 0.06049005,
 0.03024502, 0.03818913, 0.03943338, 0.03235069, 0.03531776,
 0.03684916, 0.04287902, 0.03627489, 0.04249617, 0.03407351,
 0.04077335, 0.02976646, 0.03330781, 0.03866769, 0.03416922,
 0.0395291 , 0.03196784, 0.03589204, 0.02986217, 0.03206355,
 0.03235069, 0.03885911, 0.03397779, 0.03263783, 0.0348392 ,
 0.03215926, 0.0363706 , 0.0324464 , 0.03943338, 0.03187213,
 0.02995789, 0.03062787, 0.04077335, 0.04029479, 0.03292496,
 0.03550919, 0.03072358, 0.0308193 , 0.02785222, 0.02804364,
 0.03177642, 0.02526799, 0.03713629, 0.04383614, 0.03148928,
 0.02938361, 0.02536371, 0.02459801, 0.03215926, 0.03101072,
 0.03933767, 0.0324464 , 0.04220904, 0.03656202, 0.02900077,
 0.04421899, 0.03034074, 0.02651225, 0.03292496, 0.02383231,
 0.02105666, 0.03407351, 0.02852221, 0.03828484, 0.02268377,
 0.03196784, 0.02536371, 0.02622511, 0.02737366, 0.02660796,
 0.02431087, 0.02565084, 0.02708652, 0.02402374, 0.0205781 ,
 0.02325804, 0.02545942, 0.02565084, 0.02335375, 0.01751531,
 0.02325804, 0.02813936, 0.02593798, 0.01789816, 0.02277948,
 0.02593798, 0.01837672, 0.02555513, 0.0268951 , 0.01780245,
 0.02172665, 0.01828101, 0.02383231, 0.018951 , 0.02727795,
 0.01617534, 0.01971669, 0.03503063, 0.0261294 , 0.02622511,
 0.02517228, 0.018951 , 0.02517228, 0.01942956, 0.02794793,
 0.02076953, 0.02000383, 0.01856815, 0.02622511, 0.01588821,
 0.01971669, 0.0181853 , 0.01952527, 0.0237366 , 0.01808959,
 0.01617534, 0.02392802, 0.01971669, 0.02967075, 0.02842649,
 0.02076953, 0.01397397, 0.02019525, 0.01914242, 0.01780245,
 0.02086524, 0.02000383, 0.01617534, 0.02727795, 0.01952527,
 0.0181853 , 0.01904671, 0.01550536, 0.01866386, 0.02277948,
 0.02076953, 0.01761103, 0.01684533, 0.018951 , 0.01301685,
 0.01665391, 0.01607963, 0.02038668, 0.02163093, 0.01799387,
 0.01521822, 0.01521822, 0.01780245, 0.02421516, 0.02096095,
 0.01732389, 0.01521822, 0.01904671, 0.01780245, 0.01646248,
 0.01473966, 0.0213438 , 0.01780245, 0.02574655, 0.00995406,
 0.01550536, 0.01292113, 0.01368683, 0.01473966, 0.01761103,
 0.01282542, 0.02402374, 0.01866386, 0.01550536, 0.01320827,
 0.01933384, 0.02364089, 0.01607963, 0.01560107, 0.01483538,
 0.01952527, 0.01722818, 0.01674962, 0.01512251, 0.01406968,
 0.01646248, 0.01339969, 0.01397397, 0.01847243, 0.0181853 ,
 0.01493109, 0.01320827, 0.01780245, 0.01655819, 0.01301685,
 0.01933384, 0.0142611 , 0.01837672, 0.01292113, 0.02287519,
 0.0142611 , 0.01349541, 0.01512251, 0.02019525, 0.01569678,
 0.0157925 , 0.01196401, 0.01397397, 0.01454824, 0.01301685,
 0.01837672, 0.01378254, 0.01119832, 0.0174196 , 0.01588821,
 0.01607963, 0.01378254, 0.01196401, 0.01091118, 0.01923813,
 0.01320827, 0.01445253, 0.01311256, 0.02191807, 0.0142611 ,
 0.01052833, 0.01473966, 0.01301685, 0.01215544, 0.01301685,
 0.01808959, 0.01359112, 0.01320827, 0.0142611 , 0.01272971,
 0.01244257, 0.01435681, 0.01091118, 0.00966692, 0.01808959,
 0.01521822, 0.012634 , 0.01148545, 0.01560107, 0.01177259,
 0.01215544, 0.01483538, 0.01674962, 0.01301685, 0.01713247,
 0.01205972, 0.01292113, 0.01847243, 0.01253828, 0.01234686,
 0.01550536, 0.01177259, 0.00957121, 0.01569678, 0.00985835,
 0.01205972, 0.01177259, 0.01091118, 0.01129403, 0.01330398,
 0.01033691, 0.01378254, 0.01320827, 0.01378254, 0.01196401,
 0.01387825, 0.01540965, 0.01158116, 0.01569678, 0.01234686,
 0.01043262, 0.0094755 , 0.01138974,

What does the distribution of review count across the clusters look like?

In [48]:
model.transform(data).groupBy("prediction").agg({"*": "count", "label" : "avg"}).orderBy("avg(label)").show()

+----------+--------+------------------+
prediction|count(1)| avg(label)|
+----------+--------+------------------+
 5| 14| 3.642857142857143|
 9| 260|3.7423076923076923|
 4| 12| 3.75|
 8| 270| 3.933333333333333|
 3| 1593| 3.937853107344633|
 2| 563|3.9786856127886323|
 6| 2387|3.9853372434017595|
 0| 10442| 4.013886228691821|
 7| 814| 4.022113022113022|
 1| 3604|4.6229189789123195|
+----------+--------+------------------+

Some factors besides sentiment is figuring into the clustering. But some groups are have unusual positive / negative distributions ... 

(note that due to random numbers, we may need to change the group [prediction] values below:)

In [50]:
model.transform(data).createOrReplaceTempView("clustered")

In [51]:
%sql SELECT review FROM clustered WHERE prediction = 6

review
"This mouse is amazing, I had owned a Razer Naga, and a R.A.T. 7 mouse. And I am not afraid to say that this mouse takes the cake, for $20 it looks like its worth $80. The only issue i had was that the seller sent me a wireless version, but I don't mind because it works just as well. Also where you see silver on the mouse in reality it is glossy black, and the mouse looks better IRL then in the pictures. I will add on to this review in a month or so to check in."
we recently had a baby boy so now the use of my home theater system is on the shelf for awhile. My wife also is not into big sound so decided to try out these wireless headphones. at first i almost sent them back because i took the easy route and hooked them into my receivers headphone jack. The sound was terrible and there was the dreaded hissing noise you get with some wireless electronics. before packing them up i decided to hook them up into the jacks on the TV. Difference was like night and day. the sound quality is great and really brings out some of the background sounds and music that you typically don't notice unless you are in a movie theater. They aren't super comfortable for long durations but that can be expected with most headphones. these come fully recommended.
"This is an ok pump. It has the nice bracket that can mount to your bike and you have it for the road.And although the image here doesn't show it, it does have the gauge, though it isn't the most accurate gauge (not really a problem when the tire is rated for 55 to 100 PSI).Much like one other reviewer noted, the instructions on how to switch from Schrader to Presta are not very clear and should be researched before fumbling with this blasted thing for hours trying to make it work.And like the title mentions, this isn't as tiny as the MINI name implies. When looking for a place to mount it on my bike, I am really running short on ideas. The obvious places would interfere with pedaling.All that said, it puts air in the tire. Slowly. It is double action, but the throw isn't very much. And the higher the pressure, the more difficult it is to pump."
Worked like a charm as soon as put it in my camera. I am happy with the card.
"We had a chair like this when I was a kid, only it was yellow. This one is just as good a product, with the added bonus that it's red (my favorite color)."
Most essential oils need to have a carrier oil to dilute them. Get a good book on the proper amounts for each oil you are using. This is the best carrier oil I like to use! Can use for cooking as well but I use it for it's purity in diluting my essentials!
"My puppy didn't like it very much at first. He had to get used to it. But now he loves it. It makes feeding a raw diet so easy, healthy and balanced!!"
"This was my first Bose purchase and I am glad I spent the extra money for these! Although they are not noise canceling, when I wear them they block most of my two young boys out! Enough to easily fall asleep. I listen to country music and talk radio programs; they both never sounded better. For those who like really loud music, I am sorry to say that this pair of Bose may not be for you.Shipping was great, arrived a day early and well packaged."
"I had never had a selzter bottle before. Like some others, I had started buying plain carbonated water but was appalled at how quickly the empty bottles piled up. This one fit the bill perfectly.I now charge about 1 liter a day, adding some flavorings and enjoying it quite a bit. I start with keeping the siphon in the refrigerator, along with (Brita) filtered chilled water. I find it's best to let it infuse at least an hour after charging but it is still effective if you want to use it immediately."
"Don't be fooled! These are tiny little gems, not the quarter sized ones they look like in the picture.Read thoroughly before purchasing."


Can we speculate how a group got very few reviews?

In [53]:
%sql SELECT review FROM clustered WHERE prediction = 5

review
I have a LCD Cover Screen Protector on both my camera bodies. The screen cover does make it a little harder to view my pictures in the direct sun. A second problem is that dust gets between the LCD screen and the inside of the cover. The dust requires that I remove the cover to clean it. The answer would be for the manufacture to add a gasket to the inside of their screen protector to seal it.
"I think liner is the wrong name for this product. It's more like a very thin pad. It's thicker than any other liner I've used. And it's big. Bigger than other liners too, even other ""long"" liners.It also has the signature Always ""super soft cover"" - it's kind of like a layer of mesh stuff over the pad. I hate that. The cover gives me a rash. So itchy.Most pads I've tried do have some kind of cover. But most liners don't, or they have a different kind. (Kotex for example.) Now, I know Always pads have a cover that makes me itch. I just thought the liners might be different. They are not. So if you're like me and you don't like the Always cover, you won't like these liners either."
"There are so many dog foods to choose from it's hard to know which are the best, and which dog foods you can afford. Yes, many would say ""don't own a dog if you can't feed him the best"" but not everyone can buy premium dog food. That said, I feel as though this food is higher quality than many out on the market. I like this, and I like that is higher priced but still affordable for a 70 pound dog (which I own). I am not as picky as many dog owners, but I want something our dog will eat, something that does not give her stomach problems, a food that does not give make allergies a problem, and lastly something nutritious. This seems to fit the bill. She loved the food, in fact was thrilled to get to eat at her meals. Second, she did not appear to have any problems with the food making her have stomach issues. We mixed this slow with her other food and watched during the feedings to make sure she didn't seem bound up or have loose stools. This was great! Third, our dog has problems with allergies-this didn't seem to cause any ""worse"" problems. In fact I would say that her allergies were ""better"" on this food. Lastly, I am no expert but this does appear to be a more nutritious food than many on the market. The ingredients are chicken, brown rice, vegetables, flaxseed, apples and cranberries. NO artificial colors, flavors or preservative. The company also offers a money back guarantee!"
"I purchased this stroller several years ago when my condo board threatened to make us carry our dogs while in the building because of ""accidents"" in the elevators. Well, my dog weighs 28lbs when she is on her diet but can get up to 30 lbs ( I know bad mommy too many treats!) and I am not about to carry her. Anyway, I recently started using the stroller everyday because my dog is 13 yrs old, arthritic and has a weak bladder. The board didn't follow thru with their threat to make us carry our dogs but they did implement a rule that we will be fined if our dog has an accident. So to avoid being fined, I use the Guardian Gear Sprinter EXT stroller three times a day sometimes four times a day for potty breaks. It works great! I absolutely love this stroller. It rolls over grass, cement walkways, and gravel. I live near the beach but have not tried it in sand. I love that it has springs which make it flexible. I like the zippered cover and the fact that this stroller is 21"" long so that my large Bichon can fit in the stroller. I researched a lot of strollers years ago and found this was the only one that could accommodate a 30 lb dog and is also a jogger.The front wheel locking mechanism works great and is really helpful when I do my slow, very slow jog with my dog in the stroller. The basket underneath has a tendency to slide off, but I probably should just duck tape it on since I never remove it. It is large enough to carry a few groceries or other items if you are 